In [2]:
import numpy as np

Написать программы (2 шт.), реализующие явную и неявную схемы
для уравнения

$$
\begin{array}{l}
u_t + (f(u))_x = 0 \\
u|_{t=0} = u_0(x) \\
x \in [a,b]
\end{array}
$$

Входные данные (которые могут быть изменены по указанию преподавателя):
    
    Нелинейность f (u) (для отладки можно взять f (u) = u^2);
    u0(x) (для отладки можно взять u0(x) = th(−x));
    Начало и конец отрезка и его длина;
    Краевые условия;
    Время окончания вычислений T , t ∈ [0, T ].



# Явная схема

In [5]:
def u_1_m(u, m, tau, h, F):
    return u[m] - (tau/h) * (F(u[m+1]) - F(u[m]))

def heat_expl(init, bound1, bound2, F, N, alpha, beta, tau, T):
    
    
    # YOUR CODE HERE
    w_x, h = np.linspace(alpha, beta, N+1, retstep=True)
    N_t = int(T / tau)
    u_0 = np.array([init(x) for x in w_x], dtype=np.float64)
    u_end = np.zeros(u_0.shape)
    
    for t0 in range(1, N_t+1):
        u_0 = u_end.copy()
        for m in range(1, N):
            u1m = u_1_m(u_0, m, tau, h, F)
            u1mm1 = u_1_m(u_0, m-1, tau, h, F)
            u_end[m] = 0.5 * (u_0[m] + u1m - (tau/h) * (F(u1m) - F(u1mm1)))
        u_end[0] = bound1(tau * t0)
        u_end[-1] = bound2(tau * t0)
    
    return u_end

In [8]:
def F(u):
    return u ** 2

def F_u(u):
    return 2 * u

In [7]:
from numpy.testing import assert_allclose

u_0 = heat_expl(lambda x: 0., lambda x: 1., lambda x: 1., F, 10,
                     alpha=0., beta=1., tau=0.005, T=2)

# Неявная схема

In [13]:
from scipy.linalg import solve_banded

def heat_impl(init, bound1, bound2, F, F_u, N, alpha, beta, tau, T):

    w_x, h = np.linspace(alpha, beta, N+1, retstep=True)
    N_t = int(T / tau)
    u_0 = np.array([init(x) for x in w_x], dtype=np.float64)
    u_end = np.zeros(u_0.shape)
    c = tau / (2*h)
    
    A = np.zeros((3, N - 1))
    b = np.zeros(N - 1)
    for t0 in range(1, N_t+1):
        u_0 = u_end.copy()
        u_end[0] = bound1(tau * t0)
        u_end[-1] = bound2(tau * t0)
        

        A[0] = c * F_u(u_0[1:-1])
        A[1] = 1
        A[2] = -A[0,:]
        b[0] += u_end[0] * c * F_u(u_0[1])
        b[-1] -= u_end[-1] * c * F_u(u_0[-2])
        u_end[1:-1] = solve_banded((1,1), A, b)
        
    return u_end

In [14]:
u_0 = heat_impl(lambda x: 0., lambda x: 1., lambda x: 1., F, F_u, 10,
                     alpha=0., beta=1., tau=0.005, T=2)